In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math


USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")

In [6]:
corpus_name = "cornell movie-dialogs corpus"
corpus = os.path.join("data", corpus_name)

def printLines(file, n=10):
    with open(file, 'rb') as datafile:
        lines = datafile.readlines()
    for line in lines[:n]:
        print(line)

printLines(os.path.join(corpus, "movie_lines.txt"))

b'L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!\n'
b'L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!\n'
b'L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.\n'
b'L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?\n'
b"L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go.\n"
b'L924 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Wow\n'
b"L872 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Okay -- you're gonna need to learn how to lie.\n"
b'L871 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ No\n'
b'L870 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?\n'
b'L869 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Like my fear of wearing pastels?\n'


In [7]:
# Splits each line of the file into a dictionary of fields
def loadLines(fileName, fields):
    lines = {}
    with open(fileName, 'r', encoding='iso-8859-1') as f:
        for line in f:
            values = line.split(" +++$+++ ")
            # Extract fields
            lineObj = {}
            for i, field in enumerate(fields):
                lineObj[field] = values[i]
            lines[lineObj['lineID']] = lineObj
    return lines


# Groups fields of lines from `loadLines` into conversations based on *movie_conversations.txt*
def loadConversations(fileName, lines, fields):
    conversations = []
    with open(fileName, 'r', encoding='iso-8859-1') as f:
        for line in f:
            values = line.split(" +++$+++ ")
            # Extract fields
            convObj = {}
            for i, field in enumerate(fields):
                convObj[field] = values[i]
            # Convert string to list (convObj["utteranceIDs"] == "['L598485', 'L598486', ...]")
            utterance_id_pattern = re.compile('L[0-9]+')
            lineIds = utterance_id_pattern.findall(convObj["utteranceIDs"])
            # Reassemble lines
            convObj["lines"] = []
            for lineId in lineIds:
                convObj["lines"].append(lines[lineId])
            conversations.append(convObj)
    return conversations


# Extracts pairs of sentences from conversations
def extractSentencePairs(conversations):
    qa_pairs = []
    for conversation in conversations:
        # Iterate over all the lines of the conversation
        for i in range(len(conversation["lines"]) - 1):  # We ignore the last line (no answer for it)
            inputLine = conversation["lines"][i]["text"].strip()
            targetLine = conversation["lines"][i+1]["text"].strip()
            # Filter wrong samples (if one of the lists is empty)
            if inputLine and targetLine:
                qa_pairs.append([inputLine, targetLine])
    return qa_pairs

In [8]:
# Define path to new file
datafile = os.path.join(corpus, "formatted_movie_lines.txt")

delimiter = '\t'
# Unescape the delimiter
delimiter = str(codecs.decode(delimiter, "unicode_escape"))

# Initialize lines dict, conversations list, and field ids
lines = {}
conversations = []
MOVIE_LINES_FIELDS = ["lineID", "characterID", "movieID", "character", "text"]
MOVIE_CONVERSATIONS_FIELDS = ["character1ID", "character2ID", "movieID", "utteranceIDs"]

# Load lines and process conversations
print("\nProcessing corpus...")
lines = loadLines(os.path.join(corpus, "movie_lines.txt"), MOVIE_LINES_FIELDS)
print("\nLoading conversations...")
conversations = loadConversations(os.path.join(corpus, "movie_conversations.txt"),
                                  lines, MOVIE_CONVERSATIONS_FIELDS)

# Write new csv file
print("\nWriting newly formatted file...")
with open(datafile, 'w', encoding='utf-8') as outputfile:
    writer = csv.writer(outputfile, delimiter=delimiter, lineterminator='\n')
    for pair in extractSentencePairs(conversations):
        writer.writerow(pair)

# Print a sample of lines
print("\nSample lines from file:")
printLines(datafile)



Processing corpus...

Loading conversations...

Writing newly formatted file...

Sample lines from file:
b"Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.\tWell, I thought we'd start with pronunciation, if that's okay with you.\r\n"
b"Well, I thought we'd start with pronunciation, if that's okay with you.\tNot the hacking and gagging and spitting part.  Please.\r\n"
b"Not the hacking and gagging and spitting part.  Please.\tOkay... then how 'bout we try out some French cuisine.  Saturday?  Night?\r\n"
b"You're asking me out.  That's so cute. What's your name again?\tForget it.\r\n"
b"No, no, it's my fault -- we didn't have a proper introduction ---\tCameron.\r\n"
b"Cameron.\tThe thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.\r\n"
b"The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister. 

In [9]:
# Default word tokens
PAD_token = 0  # Used for padding short sentences
SOS_token = 1  # Start-of-sentence token
EOS_token = 2  # End-of-sentence token

class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3  # Count SOS, EOS, PAD

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1

    # Remove words below a certain count threshold
    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True

        keep_words = []

        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)

        print('keep_words {} / {} = {:.4f}'.format(
            len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)
        ))

        # Reinitialize dictionaries
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3 # Count default tokens

        for word in keep_words:
            self.addWord(word)

In [11]:
MAX_LENGTH = 20  # Maximum sentence length to consider

# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s

# Read query/response pairs and return a voc object
def readVocs(datafile, corpus_name):
    print("Reading lines...")
    # Read the file and split into lines
    lines = open(datafile, encoding='utf-8').\
        read().strip().split('\n')
    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    voc = Voc(corpus_name)
    return voc, pairs

# Returns True iff both sentences in a pair 'p' are under the MAX_LENGTH threshold
def filterPair(p):
    # Input sequences need to preserve the last word for EOS token
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

# Filter pairs using filterPair condition
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

# Using the functions defined above, return a populated voc object and pairs list
def loadPrepareData(corpus, corpus_name, datafile, save_dir):
    print("Start preparing training data ...")
    voc, pairs = readVocs(datafile, corpus_name)
    print("Read {!s} sentence pairs".format(len(pairs)))
    pairs = filterPairs(pairs)
    print("Trimmed to {!s} sentence pairs".format(len(pairs)))
    print("Counting words...")
    for pair in pairs:
        voc.addSentence(pair[0])
        voc.addSentence(pair[1])
    print("Counted words:", voc.num_words)
    return voc, pairs


# Load/Assemble voc and pairs
save_dir = os.path.join("data", "save")
voc, pairs = loadPrepareData(corpus, corpus_name, datafile, save_dir)
# Print some pairs to validate
print("\npairs:")
for pair in pairs[:10]:
    print(pair)

Start preparing training data ...
Reading lines...
Read 221282 sentence pairs
Trimmed to 145439 sentence pairs
Counting words...
Counted words: 33027

pairs:
['well i thought we d start with pronunciation if that s okay with you .', 'not the hacking and gagging and spitting part . please .']
['not the hacking and gagging and spitting part . please .', 'okay . . . then how bout we try out some french cuisine . saturday ? night ?']
['you re asking me out . that s so cute . what s your name again ?', 'forget it .']
['no no it s my fault we didn t have a proper introduction', 'cameron .']
['gosh if only we could find kat a boyfriend . . .', 'let me see what i can do .']
['c esc ma tete . this is my head', 'right . see ? you re ready for the quiz .']
['that s because it s such a nice one .', 'forget french .']
['how is our little find the wench a date plan progressing ?', 'well there s someone i think might be']
['there .', 'where ?']
['you have my word . as a gentleman', 'you re sweet .']


In [36]:


import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
new_list = []
for pair in pairs:
    question_exist = False
    answer_exist = False
    sents = sent_tokenize(pair[0])
    for sentence in sents:
        if question_exist:
            break
        if len(sentence) >1:
            tokens = word_tokenize(sentence)
            for token in tokens:
                if token == '?':
                    question = sentence
                    #print(sentence)
                    question_exist = True
    sents = sent_tokenize(pair[1])
    if question_exist:
        for sentence in sents:
            if answer_exist:
                break
            if len(sentence) >1:
                tokens = word_tokenize(sentence)
                for token in tokens:
                    if token == '.':
                        answer = sentence
                        #print(sentence)
                        answer_exist = True
    if question_exist and answer_exist:
        new_list.append([question,answer])
        
                    
            




    

In [37]:
pairs = new_list
MIN_COUNT = 3    # Minimum word count threshold for trimming

def trimRareWords(voc, pairs, MIN_COUNT):
    # Trim words used under the MIN_COUNT from the voc
    voc.trim(MIN_COUNT)
    # Filter out pairs with trimmed words
    keep_pairs = []
    for pair in pairs:
        input_sentence = pair[0]
        output_sentence = pair[1]
        keep_input = True
        keep_output = True
        # Check input sentence
        for word in input_sentence.split(' '):
            if word not in voc.word2index:
                keep_input = False
                break
        # Check output sentence
        for word in output_sentence.split(' '):
            if word not in voc.word2index:
                keep_output = False
                break

        # Only keep pairs that do not contain trimmed word(s) in their input or output sentence
        if keep_input and keep_output:
            keep_pairs.append(pair)

    print("Trimmed from {} pairs to {}, {:.4f} of total".format(len(pairs), len(keep_pairs), len(keep_pairs) / len(pairs)))
    return keep_pairs


# Trim voc and pairs
pairs = trimRareWords(voc, pairs, MIN_COUNT)

keep_words 17145 / 33024 = 0.5192
Trimmed from 41392 pairs to 37880, 0.9152 of total


In [38]:
def indexesFromSentence(voc, sentence):
    return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]


def zeroPadding(l, fillvalue=PAD_token):
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

def binaryMatrix(l, value=PAD_token):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

# Returns padded input sequence tensor and lengths
def inputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar, lengths

# Returns padded target sequence tensor, padding mask, and max target length
def outputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList)
    mask = torch.BoolTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len

# Returns all items for a given batch of pairs
def batch2TrainData(voc, pair_batch):
    pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp, lengths = inputVar(input_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths, output, mask, max_target_len


# Example for validation
small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

print("input_variable:", input_variable)
print("lengths:", lengths)
print("target_variable:", target_variable)
print("mask:", mask)
print("max_target_len:", max_target_len)

input_variable: tensor([[   14,    17,    14,    89,   170],
        [  146,   357,   678,   259,    40],
        [  493,   393,   911,   177,    33],
        [   17,    69,   191,    33,     2],
        [15568,    17,    11,     2,     0],
        [  191,  3659,    33,     0,     0],
        [   84,    33,     2,     0,     0],
        [   33,     2,     0,     0,     0],
        [    2,     0,     0,     0,     0]])
lengths: tensor([9, 8, 7, 5, 4])
target_variable: tensor([[  138,   105,     4,    19,    17],
        [  296,  1440,     5,    14, 17066],
        [   88,    38,    45,   259,  8011],
        [15290,    15,   447,    15,    15],
        [ 1711,     2,    50,     2,     2],
        [   15,     0,    73,     0,     0],
        [    2,     0,  4372,     0,     0],
        [    0,     0,    15,     0,     0],
        [    0,     0,     2,     0,     0]])
mask: tensor([[ True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True],
        [ True,  True,  Tr

In [39]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding

        # Initialize GRU; the input_size and hidden_size params are both set to 'hidden_size'
        #   because our input size is a word embedding with number of features == hidden_size
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                          dropout=(0 if n_layers == 1 else dropout), bidirectional=True)

    def forward(self, input_seq, input_lengths, hidden=None):
        # Convert word indexes to embeddings
        embedded = self.embedding(input_seq)
        # Pack padded batch of sequences for RNN module
        packed = nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        # Forward pass through GRU
        outputs, hidden = self.gru(packed, hidden)
        # Unpack padding
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs)
        # Sum bidirectional GRU outputs
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        # Return output and final hidden state
        return outputs, hidden

In [40]:
# Luong attention layer
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, "is not an appropriate attention method.")
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = nn.Parameter(torch.FloatTensor(hidden_size))

    def dot_score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_output, dim=2)

    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)
        return torch.sum(hidden * energy, dim=2)

    def concat_score(self, hidden, encoder_output):
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        return torch.sum(self.v * energy, dim=2)

    def forward(self, hidden, encoder_outputs):
        # Calculate the attention weights (energies) based on the given method
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)

        # Transpose max_length and batch_size dimensions
        attn_energies = attn_energies.t()

        # Return the softmax normalized probability scores (with added dimension)
        return F.softmax(attn_energies, dim=1).unsqueeze(1)

In [41]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()

        # Keep for reference
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        # Define layers
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

        self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_step, last_hidden, encoder_outputs):
        # Note: we run this one step (word) at a time
        # Get embedding of current input word
        embedded = self.embedding(input_step)
        embedded = self.embedding_dropout(embedded)
        # Forward through unidirectional GRU
        rnn_output, hidden = self.gru(embedded, last_hidden)
        # Calculate attention weights from the current GRU output
        attn_weights = self.attn(rnn_output, encoder_outputs)
        # Multiply attention weights to encoder outputs to get new "weighted sum" context vector
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
        # Concatenate weighted context vector and GRU output using Luong eq. 5
        rnn_output = rnn_output.squeeze(0)
        context = context.squeeze(1)
        concat_input = torch.cat((rnn_output, context), 1)
        concat_output = torch.tanh(self.concat(concat_input))
        # Predict next word using Luong eq. 6
        output = self.out(concat_output)
        output = F.softmax(output, dim=1)
        # Return output and final hidden state
        return output, hidden

In [42]:
def maskNLLLoss(inp, target, mask):
    nTotal = mask.sum()
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
    loss = crossEntropy.masked_select(mask).mean()
    loss = loss.to(device)
    return loss, nTotal.item()

In [43]:
def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding,
          encoder_optimizer, decoder_optimizer, batch_size, clip, max_length=MAX_LENGTH):

    # Zero gradients
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # Set device options
    input_variable = input_variable.to(device)
    lengths = lengths.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)

    # Initialize variables
    loss = 0
    print_losses = []
    n_totals = 0

    # Forward pass through encoder
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)

    # Create initial decoder input (start with SOS tokens for each sentence)
    decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)

    # Set initial decoder hidden state to the encoder's final hidden state
    decoder_hidden = encoder_hidden[:decoder.n_layers]

    # Determine if we are using teacher forcing this iteration
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    # Forward batch of sequences through decoder one time step at a time
    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # Teacher forcing: next input is current target
            decoder_input = target_variable[t].view(1, -1)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # No teacher forcing: next input is decoder's own current output
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal

    # Perform backpropatation
    loss.backward()

    # Clip gradients: gradients are modified in place
    _ = nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    # Adjust model weights
    encoder_optimizer.step()
    decoder_optimizer.step()

    return sum(print_losses) / n_totals

In [44]:
def trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer, embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size, print_every, save_every, clip, corpus_name, loadFilename):

    # Load batches for each iteration
    training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
                      for _ in range(n_iteration)]

    # Initializations
    print('Initializing ...')
    start_iteration = 1
    print_loss = 0
    if loadFilename:
        start_iteration = checkpoint['iteration'] + 1

    # Training loop
    print("Training...")
    for iteration in range(start_iteration, n_iteration + 1):
        training_batch = training_batches[iteration - 1]
        # Extract fields from batch
        input_variable, lengths, target_variable, mask, max_target_len = training_batch

        # Run a training iteration with batch
        loss = train(input_variable, lengths, target_variable, mask, max_target_len, encoder,
                     decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size, clip)
        print_loss += loss

        # Print progress
        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(iteration, iteration / n_iteration * 100, print_loss_avg))
            print_loss = 0

        # Save checkpoint
        if (iteration % save_every == 0):
            directory = os.path.join(save_dir, model_name, corpus_name, '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size))
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save({
                'iteration': iteration,
                'en': encoder.state_dict(),
                'de': decoder.state_dict(),
                'en_opt': encoder_optimizer.state_dict(),
                'de_opt': decoder_optimizer.state_dict(),
                'loss': loss,
                'voc_dict': voc.__dict__,
                'embedding': embedding.state_dict()
            }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))

In [45]:
class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq, input_length, max_length):
        # Forward input through encoder model
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        # Prepare encoder's final hidden layer to be first hidden input to the decoder
        decoder_hidden = encoder_hidden[:decoder.n_layers]
        # Initialize decoder input with SOS_token
        decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * SOS_token
        # Initialize tensors to append decoded words to
        all_tokens = torch.zeros([0], device=device, dtype=torch.long)
        all_scores = torch.zeros([0], device=device)
        # Iteratively decode one word token at a time
        for _ in range(max_length):
            # Forward pass through decoder
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            # Obtain most likely word token and its softmax score
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            # Record token and score
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            # Prepare current token to be next decoder input (add a dimension)
            decoder_input = torch.unsqueeze(decoder_input, 0)
        # Return collections of word tokens and scores
        return all_tokens, all_scores

In [46]:
def evaluate(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):
    ### Format input sentence as a batch
    # words -> indexes
    indexes_batch = [indexesFromSentence(voc, sentence)]
    # Create lengths tensor
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    # Transpose dimensions of batch to match models' expectations
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    # Use appropriate device
    input_batch = input_batch.to(device)
    lengths = lengths.to(device)
    # Decode sentence with searcher
    tokens, scores = searcher(input_batch, lengths, max_length)
    # indexes -> words
    decoded_words = [voc.index2word[token.item()] for token in tokens]
    return decoded_words


def evaluateInput(encoder, decoder, searcher, voc):
    input_sentence = ''
    while(1):
        try:
            # Get input sentence
            input_sentence = input('> ')
            # Check if it is quit case
            if input_sentence == 'q' or input_sentence == 'quit': break
            # Normalize sentence
            input_sentence = normalizeString(input_sentence)
            # Evaluate sentence
            output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
            # Format and print response sentence
            output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
            print('Bot:', ' '.join(output_words))

        except KeyError:
            print("Error: Encountered unknown word.")

In [47]:
# Configure models
model_name = 'cb_model'
attn_model = 'dot'
#attn_model = 'general'
#attn_model = 'concat'
hidden_size = 500
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.1
batch_size = 64

# Set checkpoint to load from; set to None if starting from scratch
loadFilename = None
checkpoint_iter = 4000
#loadFilename = os.path.join(save_dir, model_name, corpus_name,
#                            '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size),
#                            '{}_checkpoint.tar'.format(checkpoint_iter))


# Load model if a loadFilename is provided
if loadFilename:
    # If loading on same machine the model was trained on
    checkpoint = torch.load(loadFilename)
    # If loading a model trained on GPU to CPU
    #checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))
    encoder_sd = checkpoint['en']
    decoder_sd = checkpoint['de']
    encoder_optimizer_sd = checkpoint['en_opt']
    decoder_optimizer_sd = checkpoint['de_opt']
    embedding_sd = checkpoint['embedding']
    voc.__dict__ = checkpoint['voc_dict']


print('Building encoder and decoder ...')
# Initialize word embeddings
embedding = nn.Embedding(voc.num_words, hidden_size)
if loadFilename:
    embedding.load_state_dict(embedding_sd)
# Initialize encoder & decoder models
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
if loadFilename:
    encoder.load_state_dict(encoder_sd)
    decoder.load_state_dict(decoder_sd)
# Use appropriate device
encoder = encoder.to(device)
decoder = decoder.to(device)
print('Models built and ready to go!')

Building encoder and decoder ...
Models built and ready to go!


In [48]:
# Configure training/optimization
clip = 50.0
teacher_forcing_ratio = 1.0
learning_rate = 0.0001
decoder_learning_ratio = 5.0
n_iteration = 4000
print_every = 1
save_every = 500

# Ensure dropout layers are in train mode
encoder.train()
decoder.train()

# Initialize optimizers
print('Building optimizers ...')
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
if loadFilename:
    encoder_optimizer.load_state_dict(encoder_optimizer_sd)
    decoder_optimizer.load_state_dict(decoder_optimizer_sd)

# If you have cuda, configure cuda to call
for state in encoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

for state in decoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

# Run training iterations
print("Starting Training!")
trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer,
           embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size,
           print_every, save_every, clip, corpus_name, loadFilename)

Building optimizers ...
Starting Training!
Initializing ...
Training...
Iteration: 1; Percent complete: 0.0%; Average loss: 9.7547
Iteration: 2; Percent complete: 0.1%; Average loss: 9.6563
Iteration: 3; Percent complete: 0.1%; Average loss: 9.4864
Iteration: 4; Percent complete: 0.1%; Average loss: 9.1962
Iteration: 5; Percent complete: 0.1%; Average loss: 8.8054
Iteration: 6; Percent complete: 0.1%; Average loss: 8.1828
Iteration: 7; Percent complete: 0.2%; Average loss: 7.8083
Iteration: 8; Percent complete: 0.2%; Average loss: 7.3456
Iteration: 9; Percent complete: 0.2%; Average loss: 7.0996
Iteration: 10; Percent complete: 0.2%; Average loss: 6.8558
Iteration: 11; Percent complete: 0.3%; Average loss: 6.6575
Iteration: 12; Percent complete: 0.3%; Average loss: 6.3212
Iteration: 13; Percent complete: 0.3%; Average loss: 6.1117
Iteration: 14; Percent complete: 0.4%; Average loss: 6.0716
Iteration: 15; Percent complete: 0.4%; Average loss: 5.9309
Iteration: 16; Percent complete: 0.4%

Iteration: 135; Percent complete: 3.4%; Average loss: 4.4353
Iteration: 136; Percent complete: 3.4%; Average loss: 4.1879
Iteration: 137; Percent complete: 3.4%; Average loss: 4.2111
Iteration: 138; Percent complete: 3.5%; Average loss: 4.3103
Iteration: 139; Percent complete: 3.5%; Average loss: 4.3656
Iteration: 140; Percent complete: 3.5%; Average loss: 4.5753
Iteration: 141; Percent complete: 3.5%; Average loss: 4.6355
Iteration: 142; Percent complete: 3.5%; Average loss: 4.4611
Iteration: 143; Percent complete: 3.6%; Average loss: 4.4734
Iteration: 144; Percent complete: 3.6%; Average loss: 4.7461
Iteration: 145; Percent complete: 3.6%; Average loss: 4.3349
Iteration: 146; Percent complete: 3.6%; Average loss: 4.5331
Iteration: 147; Percent complete: 3.7%; Average loss: 4.5675
Iteration: 148; Percent complete: 3.7%; Average loss: 4.3410
Iteration: 149; Percent complete: 3.7%; Average loss: 4.7418
Iteration: 150; Percent complete: 3.8%; Average loss: 4.6042
Iteration: 151; Percent 

Iteration: 269; Percent complete: 6.7%; Average loss: 4.2462
Iteration: 270; Percent complete: 6.8%; Average loss: 3.8437
Iteration: 271; Percent complete: 6.8%; Average loss: 4.1555
Iteration: 272; Percent complete: 6.8%; Average loss: 4.3873
Iteration: 273; Percent complete: 6.8%; Average loss: 4.0383
Iteration: 274; Percent complete: 6.9%; Average loss: 4.3125
Iteration: 275; Percent complete: 6.9%; Average loss: 4.0271
Iteration: 276; Percent complete: 6.9%; Average loss: 4.1086
Iteration: 277; Percent complete: 6.9%; Average loss: 4.1019
Iteration: 278; Percent complete: 7.0%; Average loss: 4.2321
Iteration: 279; Percent complete: 7.0%; Average loss: 3.9474
Iteration: 280; Percent complete: 7.0%; Average loss: 4.2655
Iteration: 281; Percent complete: 7.0%; Average loss: 4.0440
Iteration: 282; Percent complete: 7.0%; Average loss: 4.1757
Iteration: 283; Percent complete: 7.1%; Average loss: 4.2477
Iteration: 284; Percent complete: 7.1%; Average loss: 4.0903
Iteration: 285; Percent 

Iteration: 403; Percent complete: 10.1%; Average loss: 3.8850
Iteration: 404; Percent complete: 10.1%; Average loss: 4.0607
Iteration: 405; Percent complete: 10.1%; Average loss: 4.1413
Iteration: 406; Percent complete: 10.2%; Average loss: 3.9741
Iteration: 407; Percent complete: 10.2%; Average loss: 4.0676
Iteration: 408; Percent complete: 10.2%; Average loss: 3.9520
Iteration: 409; Percent complete: 10.2%; Average loss: 4.0362
Iteration: 410; Percent complete: 10.2%; Average loss: 4.1345
Iteration: 411; Percent complete: 10.3%; Average loss: 4.1548
Iteration: 412; Percent complete: 10.3%; Average loss: 4.0030
Iteration: 413; Percent complete: 10.3%; Average loss: 4.1884
Iteration: 414; Percent complete: 10.3%; Average loss: 4.0928
Iteration: 415; Percent complete: 10.4%; Average loss: 4.0277
Iteration: 416; Percent complete: 10.4%; Average loss: 4.1022
Iteration: 417; Percent complete: 10.4%; Average loss: 3.7378
Iteration: 418; Percent complete: 10.4%; Average loss: 4.1223
Iteratio

Iteration: 535; Percent complete: 13.4%; Average loss: 3.9563
Iteration: 536; Percent complete: 13.4%; Average loss: 3.7426
Iteration: 537; Percent complete: 13.4%; Average loss: 3.7947
Iteration: 538; Percent complete: 13.5%; Average loss: 3.9838
Iteration: 539; Percent complete: 13.5%; Average loss: 4.2340
Iteration: 540; Percent complete: 13.5%; Average loss: 4.0278
Iteration: 541; Percent complete: 13.5%; Average loss: 4.0565
Iteration: 542; Percent complete: 13.6%; Average loss: 4.0049
Iteration: 543; Percent complete: 13.6%; Average loss: 3.8639
Iteration: 544; Percent complete: 13.6%; Average loss: 4.0007
Iteration: 545; Percent complete: 13.6%; Average loss: 3.9253
Iteration: 546; Percent complete: 13.7%; Average loss: 4.0547
Iteration: 547; Percent complete: 13.7%; Average loss: 3.9852
Iteration: 548; Percent complete: 13.7%; Average loss: 3.6878
Iteration: 549; Percent complete: 13.7%; Average loss: 3.7010
Iteration: 550; Percent complete: 13.8%; Average loss: 4.0875
Iteratio

...

Iteration: 3402; Percent complete: 85.0%; Average loss: 2.9732
Iteration: 3403; Percent complete: 85.1%; Average loss: 2.7498
Iteration: 3404; Percent complete: 85.1%; Average loss: 2.7254
Iteration: 3405; Percent complete: 85.1%; Average loss: 3.0674
Iteration: 3406; Percent complete: 85.2%; Average loss: 3.1039
Iteration: 3407; Percent complete: 85.2%; Average loss: 3.1265
Iteration: 3408; Percent complete: 85.2%; Average loss: 2.8569
Iteration: 3409; Percent complete: 85.2%; Average loss: 3.0001
Iteration: 3410; Percent complete: 85.2%; Average loss: 3.0994
Iteration: 3411; Percent complete: 85.3%; Average loss: 2.9859
Iteration: 3412; Percent complete: 85.3%; Average loss: 3.0075
Iteration: 3413; Percent complete: 85.3%; Average loss: 2.8926
Iteration: 3414; Percent complete: 85.4%; Average loss: 3.0162
Iteration: 3415; Percent complete: 85.4%; Average loss: 3.0354
Iteration: 3416; Percent complete: 85.4%; Average loss: 2.8980
Iteration: 3417; Percent complete: 85.4%; Average loss:

Iteration: 3532; Percent complete: 88.3%; Average loss: 2.9249
Iteration: 3533; Percent complete: 88.3%; Average loss: 2.6501
Iteration: 3534; Percent complete: 88.3%; Average loss: 2.9270
Iteration: 3535; Percent complete: 88.4%; Average loss: 2.9630
Iteration: 3536; Percent complete: 88.4%; Average loss: 2.9170
Iteration: 3537; Percent complete: 88.4%; Average loss: 2.8534
Iteration: 3538; Percent complete: 88.4%; Average loss: 2.9915
Iteration: 3539; Percent complete: 88.5%; Average loss: 2.8960
Iteration: 3540; Percent complete: 88.5%; Average loss: 2.8683
Iteration: 3541; Percent complete: 88.5%; Average loss: 2.6194
Iteration: 3542; Percent complete: 88.5%; Average loss: 2.9589
Iteration: 3543; Percent complete: 88.6%; Average loss: 2.9766
Iteration: 3544; Percent complete: 88.6%; Average loss: 3.0824
Iteration: 3545; Percent complete: 88.6%; Average loss: 3.0408
Iteration: 3546; Percent complete: 88.6%; Average loss: 2.7352
Iteration: 3547; Percent complete: 88.7%; Average loss:

Iteration: 3662; Percent complete: 91.5%; Average loss: 3.0423
Iteration: 3663; Percent complete: 91.6%; Average loss: 3.0312
Iteration: 3664; Percent complete: 91.6%; Average loss: 2.7761
Iteration: 3665; Percent complete: 91.6%; Average loss: 2.8139
Iteration: 3666; Percent complete: 91.6%; Average loss: 2.8530
Iteration: 3667; Percent complete: 91.7%; Average loss: 2.8767
Iteration: 3668; Percent complete: 91.7%; Average loss: 2.9986
Iteration: 3669; Percent complete: 91.7%; Average loss: 2.8773
Iteration: 3670; Percent complete: 91.8%; Average loss: 2.9258
Iteration: 3671; Percent complete: 91.8%; Average loss: 2.7305
Iteration: 3672; Percent complete: 91.8%; Average loss: 2.9763
Iteration: 3673; Percent complete: 91.8%; Average loss: 2.8579
Iteration: 3674; Percent complete: 91.8%; Average loss: 2.9900
Iteration: 3675; Percent complete: 91.9%; Average loss: 2.9367
Iteration: 3676; Percent complete: 91.9%; Average loss: 2.6828
Iteration: 3677; Percent complete: 91.9%; Average loss:

Iteration: 3792; Percent complete: 94.8%; Average loss: 2.8712
Iteration: 3793; Percent complete: 94.8%; Average loss: 2.5274
Iteration: 3794; Percent complete: 94.8%; Average loss: 2.7118
Iteration: 3795; Percent complete: 94.9%; Average loss: 2.8412
Iteration: 3796; Percent complete: 94.9%; Average loss: 2.7430
Iteration: 3797; Percent complete: 94.9%; Average loss: 2.9046
Iteration: 3798; Percent complete: 95.0%; Average loss: 2.8154
Iteration: 3799; Percent complete: 95.0%; Average loss: 2.6048
Iteration: 3800; Percent complete: 95.0%; Average loss: 2.9131
Iteration: 3801; Percent complete: 95.0%; Average loss: 2.8804
Iteration: 3802; Percent complete: 95.0%; Average loss: 2.8877
Iteration: 3803; Percent complete: 95.1%; Average loss: 2.9883
Iteration: 3804; Percent complete: 95.1%; Average loss: 2.9397
Iteration: 3805; Percent complete: 95.1%; Average loss: 2.9627
Iteration: 3806; Percent complete: 95.2%; Average loss: 2.8312
Iteration: 3807; Percent complete: 95.2%; Average loss:

Iteration: 3922; Percent complete: 98.0%; Average loss: 2.6746
Iteration: 3923; Percent complete: 98.1%; Average loss: 2.7313
Iteration: 3924; Percent complete: 98.1%; Average loss: 2.7514
Iteration: 3925; Percent complete: 98.1%; Average loss: 2.8558
Iteration: 3926; Percent complete: 98.2%; Average loss: 3.0528
Iteration: 3927; Percent complete: 98.2%; Average loss: 2.8210
Iteration: 3928; Percent complete: 98.2%; Average loss: 3.0161
Iteration: 3929; Percent complete: 98.2%; Average loss: 2.9088
Iteration: 3930; Percent complete: 98.2%; Average loss: 2.9446
Iteration: 3931; Percent complete: 98.3%; Average loss: 3.1215
Iteration: 3932; Percent complete: 98.3%; Average loss: 2.8015
Iteration: 3933; Percent complete: 98.3%; Average loss: 2.6858
Iteration: 3934; Percent complete: 98.4%; Average loss: 2.8629
Iteration: 3935; Percent complete: 98.4%; Average loss: 2.7023
Iteration: 3936; Percent complete: 98.4%; Average loss: 2.4922
Iteration: 3937; Percent complete: 98.4%; Average loss:

In [ ]:
# Set dropout layers to eval mode
encoder.eval()
decoder.eval()

# Initialize search module
searcher = GreedySearchDecoder(encoder, decoder)

# Begin chatting (uncomment and run the following line to begin)
evaluateInput(encoder, decoder, searcher, voc)

> Hello
Bot: i m afraid of him . . . . . . . .
> Of who?
Bot: i don t know . to be here . . . . . .
> Where are you?
Bot: i m here . to be here . . . . . .
> What is your favorite color?
Bot: i don t know . . . . . . . .
> Do you have any hobbies?
Bot: no . . . . . . . . . .
> What do you like to do on your spare time?
Bot: i m a nun . in . . . . . . .
> You are a nun?
Bot: no . to . for a little bit . . . . .
> Where are you going?
Bot: i m going to see you . . . . . . .
> When?
Bot: i don t know . to be here . . . . . .
> What is your favorite food?
Bot: i don t know . to be here . . . . . .
> Do you like apples?
Bot: i don t know . . . . . . . .
> What do you know?
Bot: i don t know . to be here . . . . . .
